# Notebook Title

## Setup Python and R environment
you can ignore this section

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [4]:
%%R 
df <- read_csv('2023_subway_censusgeo.csv')

Rows: 6936 Columns: 43
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (19): Month, Borough_x, Equipment Type, Equipment Code, Station Name, St...
dbl (24): Total Outages, Scheduled Outages, Unscheduled Outages, Entrapments...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
df = pd.read_csv('2023_subway_censusgeo.csv')

In [6]:
df.shape

(6936, 43)

In [7]:

pd.set_option('display.max_columns', None)
df.head()


,Month,Borough_x,Equipment Type,Equipment Code,Total Outages,Scheduled Outages,Unscheduled Outages,Entrapments,Time Since Major Improvement,AM Peak Availability,AM Peak Hours Available,AM Peak Total Hours,PM Peak Availability,PM Peak Hours Available,PM Peak Total Hours,24-Hour Availability,24-Hour Hours Available,24-Hour Total Hours,Station Name,Station MRN,Station Complex Name,Station Complex MRN,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough_y,Daytime Routes,Structure,lat,long,North Direction Label,South Direction Label,ADA,ADA Notes,Georeference,GEOID,STATE,COUNTY,TRACT,BLOCK
0,01/01/2023,Brooklyn,Elevator,EL301,4,2,2,2,225.0,0.978629,121.35,124.0,0.944624,117.133333,124.0,0.977957,727.600000,744.0,ATLANTICAV-BARCLAYSCTR-4AV-D/N/R,27,"Atlantic Av (B,Q,2,3,4,5)/Pacific St (D,N,R)",617,27,617,R31,BMT,4th Av,Atlantic Av-Barclays Ctr,Bk,D N R,Subway,40.683666,-73.978810,Manhattan,Coney Island - Bay Ridge,1,NaN,POINT (-73.97881 40.683666),360470039003001,36,47,3900,3001
1,01/01/2023,Brooklyn,Elevator,EL301,4,2,2,2,225.0,0.978629,121.35,124.0,0.944624,117.133333,124.0,0.977957,727.600000,744.0,ATLANTICAV-BARCLAYSCTR-4AV-D/N/R,27,"Atlantic Av (B,Q,2,3,4,5)/Pacific St (D,N,R)",617,40,617,D24,BMT,Broadway - Brighton,Atlantic Av-Barclays Ctr,Bk,B Q,Subway,40.684460,-73.976890,Manhattan,Brighton Beach & Coney Island,1,NaN,POINT (-73.97689 40.68446),360470035001003,36,47,3500,1003
2,01/01/2023,Brooklyn,Elevator,EL301,4,2,2,2,225.0,0.978629,121.35,124.0,0.944624,117.133333,124.0,0.977957,727.600000,744.0,ATLANTICAV-BARCLAYSCTR-4AV-D/N/R,27,"Atlantic Av (B,Q,2,3,4,5)/Pacific St (D,N,R)",617,338,617,235,IRT,Eastern Pky,Atlantic Av-Barclays Ctr,Bk,2 3 4 5,Subway,40.684359,-73.977666,Manhattan,Flatbush - New Lots,1,NaN,POINT (-73.977666 40.684359),360470035001003,36,47,3500,1003
3,01/01/2023,Brooklyn,Elevator,EL709,6,2,4,0,145.0,0.985081,122.15,124.0,0.974866,120.883333,124.0,0.971214,722.583333,744.0,JAYST-METROTECH-BWY-R,25,"Jay St-MetroTech (A,C,F,R)",636,25,636,R29,BMT,Broadway,Jay St-MetroTech,Bk,R,Subway,40.692180,-73.985942,Manhattan,Bay Ridge - 95 St,1,NaN,POINT (-73.985942 40.69218),360470011001011,36,47,1100,1011
4,01/01/2023,Brooklyn,Elevator,EL709,6,2,4,0,145.0,0.985081,122.15,124.0,0.974866,120.883333,124.0,0.971214,722.583333,744.0,JAYST-METROTECH-BWY-R,25,"Jay St-MetroTech (A,C,F,R)",636,174,636,A41,IND,8th Av - Fulton St,Jay St-MetroTech,Bk,A C F,Subway,40.692338,-73.987342,Manhattan,Euclid - Lefferts - Rockaways - Coney Island,1,NaN,POINT (-73.987342 40.692338),360470011001001,36,47,1100,1001


## 👉 Grab Census Data

1. loading the Census API key

In [8]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()


True

In [9]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("CENSUS_API_KEY"))

Loading required package: tidycensus
To install your API key for use in future sessions, run this function with `install = TRUE`.


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


In [22]:
# %%R 

# Finding the Census Varaibles for the ACS 5 year survey
# Generally you'd do this in CensusReporter, but since it's down sometimes, here it is using tidycensus's load_variables function

# get every single variable in the ACS5
all_census_vars <- load_variables(2021, "acs5", cache = TRUE) 

filtered_census_vars <- all_census_vars %>% 
    filter(grepl("median income", label, ignore.case = TRUE))   # filter to those containing "median income"
    
# write to CSV so we can view it in python
filtered_census_vars %>% 
    write_csv("filtered_census_vars.csv")

# show the first few rows
filtered_census_vars %>%
    select(-geography) %>% # remove the geography column
    print(n = 20) # print the first 20 rows

SyntaxError: invalid syntax (3197094227.py, line 9)

In [10]:
%%R 
# the variable B01003_001E was selectd from the census table 
# for population, which we found in censusreporter here:
# https://censusreporter.org/tables/B01003/

# in the table below, pick the geography, the variables, and the survey you want to pull from
# see the possible values here https://walker-data.com/tidycensus/articles/basic-usage.html

# Get variable from ACS
nyc_census_data <- get_acs(geography = "tract", 
                      state='NY',
                      county = c("New York", "Kings", "Queens", "Bronx", "Richmond"),
                      variables = c(
                        population="B01003_001", 
                        med_earn="B19013_001", # Median household income in the past 12 months
                        sub_pop='B08301_012', # Population using subway or elevated rail to work
                        amb_pop='B18105_001' # Population with Ambulatory Difficulty
                      ), 
                      year = 2021,
                      survey="acs5",
                      geometry=T)
options(width = 1000)

nyc_census_data

  |======================================================================| 100%
Simple feature collection with 9308 features and 5 fields (with 4 geometries empty)
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -74.25609 ymin: 40.4961 xmax: -73.70036 ymax: 40.91771
Geodetic CRS:  NAD83
First 10 features:
         GEOID                                       NAME   variable estimate   moe                       geometry
1  36081014700  Census Tract 147, Queens County, New York population     2863   513 MULTIPOLYGON (((-73.9137 40...
2  36081014700  Census Tract 147, Queens County, New York    sub_pop      824   210 MULTIPOLYGON (((-73.9137 40...
3  36081014700  Census Tract 147, Queens County, New York    amb_pop     2738   508 MULTIPOLYGON (((-73.9137 40...
4  36081014700  Census Tract 147, Queens County, New York   med_earn    71815 18034 MULTIPOLYGON (((-73.9137 40...
5  36047058400   Census Tract 584, Kings County, New York population     3655   529 MULTIPOLYGON ((

Getting data from the 2017-2021 5-year ACS
Using FIPS code '36' for state 'NY'
Using FIPS code '061' for 'New York County'
Using FIPS code '047' for 'Kings County'
Using FIPS code '081' for 'Queens County'
Using FIPS code '005' for 'Bronx County'
Using FIPS code '085' for 'Richmond County'


## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [11]:
%%R

# pivot from long to wide
nyc_census_data <- nyc_census_data %>% 
  pivot_wider(
    names_from = variable, 
    values_from = c(estimate, moe),
    names_glue = "{variable}_{.value}"
  )
options(width = 1000)
nyc_census_data

Simple feature collection with 2327 features and 10 fields (with 1 geometry empty)
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -74.25609 ymin: 40.4961 xmax: -73.70036 ymax: 40.91771
Geodetic CRS:  NAD83
# A tibble: 2,327 × 11
   GEOID       NAME                                                                                                                       geometry population_estimate sub_pop_estimate amb_pop_estimate med_earn_estimate population_moe sub_pop_moe amb_pop_moe med_earn_moe
   <chr>       <chr>                                                                                                            <MULTIPOLYGON [°]>               <dbl>            <dbl>            <dbl>             <dbl>          <dbl>       <dbl>       <dbl>        <dbl>
 1 36081014700 Census Tract 147, Queens County, New York   (((-73.9137 40.76548, -73.9121 40.76473, -73.9113 40.76435, -73.9105 40.76398, -73.9...                2863              824             2738         

In [12]:
%%R
df

# A tibble: 6,936 × 43
   Month      Borough_x `Equipment Type` `Equipment Code` `Total Outages` `Scheduled Outages` `Unscheduled Outages` Entrapments `Time Since Major Improvement` `AM Peak Availability` `AM Peak Hours Available` `AM Peak Total Hours` `PM Peak Availability` `PM Peak Hours Available` `PM Peak Total Hours` `24-Hour Availability` `24-Hour Hours Available` `24-Hour Total Hours` `Station Name`                   `Station MRN` `Station Complex Name`                       `Station Complex MRN` `Station ID` `Complex ID` `GTFS Stop ID` Division Line                `Stop Name`              Borough_y `Daytime Routes` Structure   lat  long `North Direction Label` `South Direction Label`                        ADA `ADA Notes` Georeference                   GEOID STATE COUNTY TRACT  BLOCK
   <chr>      <chr>     <chr>            <chr>                      <dbl>               <dbl>                 <dbl>       <dbl>                          <dbl>                  <dbl>                

In [13]:
%%R 

# keep the first 11 digits in df$GEOID
df$GEOID <-  substr(df$GEOID, 1, 11) %>%
    as.numeric(df$GEOID)

# change df$GEOID to double
nyc_census_data$GEOID <- as.numeric(nyc_census_data$GEOID)
df$GEOID <- as.numeric(df$GEOID)
    
# merge nyc_census_data with df on GEOID
df_census <- merge(df, nyc_census_data, by = "GEOID")

In [1]:
%%R 
df_census

UsageError: Cell magic `%%R` not found.


In [15]:
%%R 
write_csv(df_census, "2023_subway_censusvar.csv")